In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import os
import PIL
import PIL.Image
import pathlib
import math
import copy

## Preprocessing

In [3]:
 def downscale_images(images, scale=30):
    """
    Downscales images in an array.
    Args:
        images: array of images to be downscaled 
        scale: 

    Returns: array of downscaled down images

    """
    resized = []
    
    for img in images:
        
        width = int(img.shape[1] * scale / 100)
        height = int(img.shape[0] * scale / 100)
        dim = (width, height)

        resized.append(cv2.resize(img, dim, interpolation = cv2.INTER_AREA))
    
    return resized


In [4]:
def load_and_preprocess(path):
    
    
    # directory with the images
    data_dir = pathlib.Path(path)
    
    
    # how many images are in the directory
    image_count = len(list(data_dir.glob('*')))
    print(f"Number of images found in {path}: {image_count}")


    paintings_path = list(data_dir.glob('*'))
    paintings_path = [str(path) for path in paintings_path]
    
    # sort by img number
    paintings_path = sorted(paintings_path, key=lambda path: int(path.split('_')[-1].split('.')[0]))
    
    
    # loading images, in BGR
    paintings = [cv2.imread(painting_path) for painting_path in paintings_path]
    paintings = downscale_images(paintings)
    
    
    # converting to gray
    paintings_gray = [cv2.cvtColor(painting, cv2.COLOR_BGR2GRAY) for painting in paintings]
    
    # converting to RGB for later visualisation
    paintings_rgb = [cv2.cvtColor(painting, cv2.COLOR_BGR2RGB) for painting in paintings]
    
    painting_tuple = (paintings_gray, paintings_rgb)
    
    return painting_tuple


## Loading stolen paintings

In [5]:
stolen_art_dir_path = "../datasets/Tate-500-wop-opac/"
stolen_paintings = load_and_preprocess(stolen_art_dir_path)


Number of images found in ../datasets/Tate-500-wop-opac/: 477


## Loading found paintings
Paintings which I want to test against the stolen images.

In [6]:
found_paintings_dir_path = "../augmentation/Tate-500-wop-opac_aug/" 
found_paintings = load_and_preprocess(found_paintings_dir_path)

Number of images found in ../augmentation/Tate-500-wop-opac_aug/: 477


In [7]:
def display_compared(img1, img2):
    # Display traning image and testing image
    fx, plots = plt.subplots(1, 2, figsize=(20,10))

    plots[0].set_title("Stolen Image")
    plots[0].imshow(img1)

    plots[1].set_title("Comparing to")
    plots[1].imshow(img2)

## Scaling functions for visualisation

Functions mainly for downscaling the paintings/keypoints before visualisation. Greatly reduces the size of the notebook.

In [8]:
def features_deepcopy (f):
    return [cv2.KeyPoint(x = k.pt[0], y = k.pt[1], 
            _size = k.size, _angle = k.angle, 
            _response = k.response, _octave = k.octave, 
            _class_id = k.class_id) for k in f]

In [9]:
def scale_keypoints(keypoints, scale):
    
    for keypoint in keypoints:
        new_x = math.trunc(keypoint.pt[0]*(scale/100) )
        new_y = math.trunc(keypoint.pt[1]*(scale/100) )
        keypoint_scaled = (new_x, new_y) 
        keypoint.pt = keypoint_scaled
    

In [10]:
def draw_matches_scaled(found_painting, found_painting_keypoints,
                        stolen_painting, stolen_painting_keypoints,
                        matches, matchesMask, draw_params, scale = 20):
    
    # scale the paintings
    found_painting = downscale_images([found_painting], scale)[0]
    stolen_painting = downscale_images([stolen_painting], scale)[0]
    
    found_painting_keypoints_scaled = features_deepcopy(found_painting_keypoints)
    scale_keypoints(found_painting_keypoints_scaled, scale)
    
    stolen_painting_keypoints_scaled = features_deepcopy(stolen_painting_keypoints)
    scale_keypoints(stolen_painting_keypoints_scaled, scale)

    result = cv2.drawMatchesKnn(found_painting, found_painting_keypoints_scaled,
                                stolen_painting, stolen_painting_keypoints_scaled,
                                matches, None, **draw_params)
    
    return result

    

# ORB

In [11]:
# Creating an ORB object
orb = cv2.ORB_create(nfeatures=500)

In [12]:
def visualize_keypoints(image, keypoints, scale = 100):
    
    # SCALE HAS TO BE 100, TODO
    
    keypoints_without_size = np.copy(image)
    keypoints_with_size = np.copy(image)

    keypoints_without_size = downscale_images([keypoints_without_size], scale)[0]
    keypoints_with_size = downscale_images([keypoints_with_size], scale)[0]
    
    keypoints_scaled = features_deepcopy(keypoints)
    scale_keypoints(keypoints_scaled, scale)
    
    cv2.drawKeypoints(image, keypoints_scaled, keypoints_without_size, color = (0, 255, 0))

    cv2.drawKeypoints(image, keypoints_scaled, keypoints_with_size, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    
    # Display image with and without keypoints size
    fx, plots = plt.subplots(1, 2, figsize=(20,10))

    plots[0].set_title("Train keypoints With Size")
    plots[0].imshow(keypoints_with_size)

    plots[1].set_title("Train keypoints Without Size")
    plots[1].imshow(keypoints_without_size)
    
    print("Number of Keypoints Detected In The Stolen Image: ", len(keypoints))

## Running the algorithm

Find keypoints on every image in the database. Display the keypoints with their size (left) and without (right).

In [13]:
orb_results = []

for index, _ in enumerate(stolen_paintings[0]):

    orb_res = orb.detectAndCompute(stolen_paintings[0][index], None)
    orb_results.append(orb_res)

    sp_keypoints, sp_descriptors = orb_res
    
    #visualize_keypoints(stolen_paintings[1][index], sp_keypoints, scale=100)

# Searching for a painting in the database

In [14]:
def search_for_painting(painting_index, draw_match = True):
    
    found_painting = found_paintings[0][painting_index]
    found_painting_rgb = found_paintings[1][painting_index]
    found_painting_keypoints, found_painting_descriptor = orb.detectAndCompute(found_painting, None)

    best_match_index, matches, mask = get_best_match(found_painting_descriptor)
    
    if best_match_index < 0 and draw_match:
        print(f"Sorry, no match found for painting: {painting_index}")
        # TODO - fix imshow
        plt.imshow(found_paintings[1][painting_index])
        return
    
    if draw_match:
        draw_matches(best_match_index, found_painting_rgb, found_painting_keypoints, matches, mask)
    
    return best_match_index
    

## Techniques to improve accuracy

### Histogram comparison

In [15]:
from scipy.spatial import distance as dist

def calculate_histograms(images, bin_size = 32):
    
    hists = [cv2.calcHist([image], [0, 1, 2], None, [bin_size, bin_size, bin_size], [0, 256, 0, 256, 0, 256])
             for image in images]
    
    hists = [cv2.normalize(hist, hist).flatten() for hist in hists]
    
    return hists

In [16]:
found_paintings_hists = calculate_histograms(found_paintings[1])
stolen_paintings_hists = calculate_histograms(stolen_paintings[1])

In [32]:
def compare_hists(hist1, hist2):
    
    corr = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    
    return corr

## FLANN Based Matcher

In [18]:
# 1. parametr FlannBasedMatcheru
FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6, # 12
                   key_size = 12,     # 20
                   multi_probe_level = 1) #2

In [19]:
# 2. parametr FlannBasedMatcheru
search_params = dict(checks=50)   # or pass empty dictionary

In [20]:
def get_best_match(found_painting_descriptor, k = 2, min_matches = 10):

    matches_count = {}
    matches_count[-1] = 0
    
    matches = {}
    matches[-1] = None
    
    matches_masks = {}
    matches_masks[-1] = None
    
    
    for index, stolen_res in enumerate(orb_results):
        stolen_keypoints, stolen_descriptor = stolen_res

        # TODO - this was lazy fixed
        if found_painting_descriptor is None or len(found_painting_descriptor) <= k or stolen_descriptor is None or len(stolen_descriptor) <= k:
            best_match_index = -1
            break
        
        # Cross check parametr
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # Perform the matching between the ORB descriptors of the training image and the test image
        matches[index] = flann.knnMatch(stolen_descriptor, found_painting_descriptor, k)
        # deskriptor = "fingerprint" keypointu, vektor 0 a 1, napr. BRIEF rozmaze misto a z toho spocita vektor
        # ORB muze pouzivat rBRIEF, tzn. pocita i s rotaci obrazu

        ok_matches_num = 0

        # Need to draw only good matches, so create a mask
        matches_masks[index] = [[0,0] for i in range(len(matches[index]))]
        # ratio test as per Lowe's paper
        for i, candidates in enumerate(matches[index]):
            if (len(candidates)<2):
                continue # nedostatek bodu pro porovnani
            m, n = candidates    
            if m.distance < 0.7*n.distance:
                matches_masks[index][i]=[1,0]
                ok_matches_num = ok_matches_num + 1 
        
        matches_count[index] = ok_matches_num
            
        
        # print("Pocet prijatelnych matchu: ", ok_matches_num)
        
    best_match_index = max(matches_count, key=matches_count.get)
    #print(type(best_match_index))
    best_match_index = best_match_index if matches_count[best_match_index] >= min_matches else -1
        
    return best_match_index, matches[best_match_index], matches_masks[best_match_index] 

In [21]:
def draw_matches(stolen_index, found_painting, found_painting_keypoints, matches, matchesMask):
    
    draw_params = dict(matchColor = (0,255,0),
                       singlePointColor = (255,0,0),
                       matchesMask = matchesMask,
                       flags = cv2.DrawMatchesFlags_DEFAULT)

    
    
    result = draw_matches_scaled(found_painting, found_painting_keypoints,
                                 stolen_paintings[1][stolen_index], orb_results[stolen_index][0],
                                 matches, matchesMask, draw_params, 100)
    
    # Display the best matching points
    plt.rcParams['figure.figsize'] = [28.0, 14.0]
    plt.title('Painting found! Best match: ')
    plt.imshow(result)
    plt.show()

# Testing the algorithm on all the paintings

In [33]:
total_searched = len(found_paintings[0])
not_found = {}
found_ok = {}
found_wrong = {}

histograms_good_match = []
histograms_bad_match = []

for index in range(0, total_searched):
    best_match_index = search_for_painting(index, draw_match = False)
    
    print(f"Best match for {index} is {best_match_index}")
    
    hist_corr = 0
    if best_match_index is not None:
        hist_corr = compare_hists(found_paintings_hists[index], stolen_paintings_hists[best_match_index])
        print(f"Histogram correlation: {hist_corr}")
    
    if index == best_match_index:
        histograms_good_match.append(hist_corr)
    else:
        histograms_bad_match.append(hist_corr)
    
    if index is None:
        not_found[index] = best_match_index
    elif index == best_match_index:
        found_ok[index] = best_match_index
    elif index != best_match_index:
        found_wrong[index] = best_match_index



Best match for 0 is 0
Histogram correlation: 0.039568990976050336
Best match for 1 is 1
Histogram correlation: 0.7528875159207026
Best match for 2 is 339
Histogram correlation: 0.007662684952125709
Best match for 3 is 284
Histogram correlation: 0.23658478272786884
Best match for 4 is 312
Histogram correlation: 0.0008676106516836554
Best match for 5 is 285
Histogram correlation: -0.0011879801035128224
Best match for 6 is 6
Histogram correlation: 0.533325176894145
Best match for 7 is 7
Histogram correlation: 0.11409113766575715
Best match for 8 is 8
Histogram correlation: 0.7321931900255695
Best match for 9 is 9
Histogram correlation: 0.6516108677414055
Best match for 10 is 221
Histogram correlation: 0.4994769345134565
Best match for 11 is 11
Histogram correlation: 0.0007186062276602036
Best match for 12 is 346
Histogram correlation: -0.002261388218274785
Best match for 13 is 314
Histogram correlation: 0.06515690201106426
Best match for 14 is 14
Histogram correlation: 0.6304667873941563


Best match for 121 is 89
Histogram correlation: 0.0721440420744795
Best match for 122 is 4
Histogram correlation: 0.016262352962914232
Best match for 123 is 374
Histogram correlation: 0.0709558230625793
Best match for 124 is -1
Histogram correlation: -4.217711165165961e-05
Best match for 125 is 125
Histogram correlation: 0.0005138709064525826
Best match for 126 is 4
Histogram correlation: 0.05071245868487772
Best match for 127 is 173
Histogram correlation: 0.05790342035880846
Best match for 128 is 128
Histogram correlation: 0.694243709288739
Best match for 129 is 339
Histogram correlation: 7.905593008057683e-05
Best match for 130 is 130
Histogram correlation: 0.031055974377804458
Best match for 131 is 131
Histogram correlation: 0.12355870220090387
Best match for 132 is 112
Histogram correlation: 0.024420709574869304
Best match for 133 is 39
Histogram correlation: 0.2857824887271495
Best match for 134 is -1
Histogram correlation: 0.005002672666585733
Best match for 135 is 249
Histogram 

Best match for 240 is 383
Histogram correlation: 0.2660007370164306
Best match for 241 is 236
Histogram correlation: 0.05660012389501819
Best match for 242 is 71
Histogram correlation: 0.0026838299354426977
Best match for 243 is -1
Histogram correlation: 0.02732603752957438
Best match for 244 is 244
Histogram correlation: 0.4659140549590757
Best match for 245 is 245
Histogram correlation: 0.22379967857565003
Best match for 246 is 388
Histogram correlation: 0.0029694342615080526
Best match for 247 is 247
Histogram correlation: 0.504464416152261
Best match for 248 is 371
Histogram correlation: 0.15374194566625587
Best match for 249 is 249
Histogram correlation: 0.9721129017481303
Best match for 250 is 250
Histogram correlation: 0.22404651340700915
Best match for 251 is 312
Histogram correlation: -0.00022624512193439173
Best match for 252 is 270
Histogram correlation: 0.03946428259248494
Best match for 253 is -1
Histogram correlation: 0.04889765818778967
Best match for 254 is -1
Histogram

Best match for 359 is 24
Histogram correlation: 0.341957029242277
Best match for 360 is -1
Histogram correlation: 0.18772120278277243
Best match for 361 is 361
Histogram correlation: 0.16153696102782567
Best match for 362 is 100
Histogram correlation: 0.004408439312112982
Best match for 363 is 19
Histogram correlation: 0.025511948438806234
Best match for 364 is 364
Histogram correlation: 0.5609844497812833
Best match for 365 is 322
Histogram correlation: 0.00023133848197310274
Best match for 366 is 386
Histogram correlation: 0.17535783137183553
Best match for 367 is 367
Histogram correlation: 0.011303181259366874
Best match for 368 is 368
Histogram correlation: 0.692721567209443
Best match for 369 is 214
Histogram correlation: 0.013873991525045665
Best match for 370 is 154
Histogram correlation: 0.21451558926113448
Best match for 371 is 329
Histogram correlation: 0.30620635538279883
Best match for 372 is 117
Histogram correlation: -0.0004747700086767284
Best match for 373 is 373
Histog

In [34]:
accuracy = len(found_ok) / (len(not_found) + len(found_wrong))
print(f"Accuracy: {accuracy*100} %")

Accuracy: 63.91752577319587 %


#### Histograms vs ORB analysis
How correlated were the histograms depending on whether ORB matched them correctly or not?

In [40]:
import pandas as pd
import numpy as np


#print(hist_good_match)
hists_g = pd.DataFrame(histograms_good_match).rename(columns={0:"Good"})
print(hists_g.describe())

hists_b = pd.DataFrame(histograms_bad_match).rename(columns={0:"Bad"})
print(hists_b.describe())

             Good
count  186.000000
mean     0.381348
std      0.302769
min      0.000057
25%      0.098480
50%      0.337458
75%      0.630776
max      0.981539
              Bad
count  291.000000
mean     0.080117
std      0.144725
min     -0.002689
25%      0.001844
50%      0.019449
75%      0.075423
max      0.834984


### Visualizing results

In [24]:
def visualize_image_tuples(index_dict):
    
    for index_searched, index_found in index_dict.items():
        
        fx, plots = plt.subplots(1, 2, figsize=(20,10))
    
        plots[0].set_title("Searched image:")
        plots[0].imshow(found_paintings[1][index_searched])

        plots[1].set_title("Wrongly matched with:")
        plots[1].imshow(stolen_paintings[1][index_found])


In [ ]:
visualize_image_tuples(found_wrong)

<ipython-input-24-ea87fab9a572>:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fx, plots = plt.subplots(1, 2, figsize=(20,10))


## Matching nalezeného obrazu s obrazy v DB  pomocí Brute Force Matcher


BFMatcher - Obsahuje parametr crossCheck, údajně dobrá alternativa k ratio testu, ale mně nefunguje dobře. Znamená to, že namatchuje pouze body, které jsou si nejpodobnější navzájem, tzn. pro žádný z dvojice neexistuje v druhém obraze lepší match. Pokud ho dám na False, matcher namatchuje všechny body a výsledek je uplně k ničemu. Pokud na true, matcher namatchuje cca polovinu bodů, ale u všech obrazů a je proto taky k ničemu.

Mnohem lépe se mi osvědčila kombinace knnMatch a ratio testu.

**crossCheck se samozřejmě nedá kombinovat s knnMatch ! !**




zdroj: https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html

In [ ]:
kbfm = 2

for index, stolen_res in enumerate(orb_results):
    stolen_keypoints, stolen_descriptor = stolen_res

    # Cross check parametr
    bf = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck = False)

        
    
    # Perform the matching between the ORB descriptors of the training image and the test image
    matches = bf.knnMatch(stolen_descriptor, suspect_descriptor, kbfm)
    # deskriptor = "fingerprint" keypointu, vektor 0 a 1, napr. BRIEF rozmaze misto a z toho spocita vektor
    # ORB muze pouzivat rBRIEF, tzn. pocita i s rotaci obrazu
    
   
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])

    result = cv2.drawMatchesKnn(stolen_paintings[1][index], stolen_keypoints, suspected_painting_rgb, suspect_keypoints, good, None, flags = 2)
    
    # Display the best matching points
    plt.rcParams['figure.figsize'] = [28.0, 14.0]
    plt.title('Best Matching Points')
    plt.imshow(result)
    plt.show()

    # Print total number of matching points between the training and query images
    print("Celkový počet dobrých match (po ratio testu): ", len(good))